<a href="https://colab.research.google.com/github/nipunnirmal21/Z-score-based-university-course-recommendation-system/blob/main/Z_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [3]:
try:
    df = pd.read_csv('university_dataset.csv')
    print("Dataset loaded successfully!")
    print(f"\nTotal data points: {len(df)}")
except FileNotFoundError:
    print("Error: 'university_dataset.csv' not found. Please ensure it's in the correct location.")
    # Exit or handle the error appropriately if the file is crucial
    exit()

Dataset loaded successfully!

Total data points: 1575


In [4]:
df['Stream'] = df['Stream'].str.strip().str.lower()
df['District'] = df['District'].str.strip().str.lower()
df['Degree Program'] = df['Degree Program'].str.strip().str.lower()

In [5]:
unique_streams = df['Stream'].unique().tolist()
unique_districts = df['District'].unique().tolist()
unique_degree_programs = df['Degree Program'].unique().tolist()

In [6]:
encoder_stream = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_streams = encoder_stream.fit_transform(df[['Stream']])
encoded_stream_df = pd.DataFrame(encoded_streams, columns=encoder_stream.get_feature_names_out(['Stream']))

In [7]:
encoder_district = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_districts = encoder_district.fit_transform(df[['District']])
encoded_district_df = pd.DataFrame(encoded_districts, columns=encoder_district.get_feature_names_out(['District']))

In [8]:
df_processed = pd.concat([df[['Min Z-score']], encoded_stream_df, encoded_district_df], axis=1)


In [11]:
def recommend_courses_rule_based(z_score, stream, district, dataset):
    """
    Recommends university degree programs based on Z-score, stream, and district
    using a rule-based filtering approach.

    Args:
        z_score (float): The student's Z-score.
        stream (str): The student's A/L stream (e.g., 'physical science').
        district (str): The student's district (e.g., 'colombo').
        dataset (pd.DataFrame): The DataFrame containing university admission data.

    Returns:
        pd.DataFrame: A DataFrame of recommended courses.
    """

    stream = stream.strip().lower()
    district = district.strip().lower()

    recommended = dataset[
        (dataset['Stream'] == stream) &
        (dataset['District'] == district) &
        (dataset['Min Z-score'] <= z_score)
    ]

    recommended = recommended.sort_values(by='Min Z-score', ascending=False)
    return recommended[['Degree Program', 'Stream', 'District', 'Min Z-score']]
